<a href="https://colab.research.google.com/github/srishti-git1110/Hugging-Face-Transformers/blob/main/Text_Classifier_%F0%9F%A4%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook illustrates simple text classification using distillBERT WITHOUT fine tuning the parameters of the pretrained transformers.

# We will just use the hidden embedding associated with cls to train a logistic classifier & a neural network based classifier.

In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 40.1 MB/s 
     |████████████████████████████████| 6.6 MB 30.5 MB/s 
     |████████████████████████████████| 895 kB 33.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 38.5 MB/s 
     |████████████████████████████████| 212 kB 43.1 MB/s 
     |████████████████████████████████| 136 kB 47.1 MB/s 
     |████████████████████████████████| 127 kB 44.9 MB/s 
     |████████████████████████████████| 271 kB 34.4 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 48.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [2]:
from transformers import AutoModel, AutoTokenizer
import torch
model_ckpt = "distilbert-base-uncased"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt).to(device)

#getting the last hidden states - illustration for a single line of text
text = "This is a car"
encoded_text = tokenizer(text, return_tensors='pt')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
encoded_text

{'input_ids': tensor([[ 101, 2023, 2003, 1037, 2482,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

Size of input_ids = batch size, no of tokens

batch size = no of sentences basically no of elements in a list of strings



In [4]:
inputs = {k:v.to(device) for k,v in encoded_text.items()}     #mark here encoded text is a dict
inputs

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0'),
 'input_ids': tensor([[ 101, 2023, 2003, 1037, 2482,  102]], device='cuda:0')}

In [5]:
#ignore this. this is just to illustrate the sentences aren't separated by . automatically

text = "this is a car. it's from tesla"
encoded_text = tokenizer(text, return_tensors='pt')
encoded_text

{'input_ids': tensor([[  101,  2023,  2003,  1037,  2482,  1012,  2009,  1005,  1055,  2013,
         26060,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
with torch.no_grad():
 outputs = model(**inputs)
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.1674, -0.1444,  0.1681,  ..., -0.2169,  0.2481,  0.3161],
         [-0.3775, -0.3395,  0.1117,  ..., -0.6439,  0.7683,  0.1666],
         [-0.3175, -0.4233,  0.2345,  ..., -0.2955,  0.4007,  0.6818],
         [-0.3386, -0.2318,  0.4602,  ..., -0.4317,  0.2301,  0.6278],
         [ 0.2979, -0.1982,  0.0905,  ..., -0.4294, -0.0102, -0.0466],
         [ 0.7991,  0.0667, -0.3333,  ...,  0.0248, -0.5705, -0.3411]]],
       device='cuda:0'), hidden_states=None, attentions=None)


In [7]:
from datasets import load_dataset
emotions = load_dataset("emotion")

Using custom data configuration default


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
print(emotions['train'].features) 

#we encode text
data_to_be_encoded = emotions['train'][:2]['text']  #[:2]
encoded_data = tokenizer(data_to_be_encoded, return_tensors="pt", padding=True, truncation=True)

print(data_to_be_encoded)
print(encoded_data)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=6, names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}
['i didnt feel humiliated', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake']
{'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  101,  1045,  2064,  2175,  2013,  3110,  2061, 20625,  2000,  2061,
          9636, 17772,  2074,  2013,  2108,  2105,  2619,  2040, 14977,  1998,
          2003,  8300,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [9]:
encoded_data = {k:v.to(device) for k,v in encoded_data.items()}

In [10]:
with torch.no_grad():
 outputs = model(**encoded_data)
print(outputs)


#Note what goes inside model(), its the encoded sentences - the input ids and attention masks in tensor forms

BaseModelOutput(last_hidden_state=tensor([[[-1.1675e-01,  9.8571e-02, -1.2963e-01,  ...,  5.8711e-02,
           3.5433e-01,  4.0421e-01],
         [ 1.3251e-01,  1.5157e-01, -1.1694e-01,  ..., -1.1192e-01,
           5.5620e-01,  2.9076e-01],
         [-1.0527e-01,  2.8617e-01,  1.9578e-01,  ...,  2.4091e-02,
           5.7729e-02, -3.6270e-01],
         ...,
         [-1.1104e-01,  1.2956e-01, -8.7955e-05,  ...,  1.7412e-01,
           7.5147e-02, -2.0194e-01],
         [-8.6914e-02,  1.2423e-01,  2.0505e-03,  ...,  1.7200e-01,
           8.7336e-02, -1.9188e-01],
         [-4.0857e-02,  9.5340e-02, -6.3924e-02,  ...,  1.6551e-01,
           8.2269e-02, -1.4711e-01]],

        [[-3.2363e-02, -3.2315e-02, -1.9573e-01,  ..., -1.7466e-01,
           3.5464e-01,  3.0277e-01],
         [ 2.9204e-01,  2.5143e-01, -3.5096e-01,  ..., -7.0466e-02,
           4.4761e-01,  3.4334e-01],
         [ 1.1239e-01,  2.7683e-01, -1.3712e-01,  ..., -3.5484e-01,
           6.6407e-01,  9.7670e-02],
     

In [11]:
outputs.last_hidden_state.shape

torch.Size([2, 23, 768])

last hidden state dim = batch size, no of tokens, dim of hidden vector

In [12]:
print(emotions)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [13]:
#tokenizing the whole dataset

def tokenize(batch):
 return tokenizer(batch["text"], padding=True, truncation=True) # here you cant set return_tensors=True

emotions_encoded = emotions.map(tokenize, batch_size=None, batched=True)  # batch_size=None, tokenize() function will be applied on the full dataset as a single batch
emotions_encoded

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [14]:
# getting last hidden states of whole dataset

def extract_hidden_states(batch):
  """ puts input dict values to device, and calculates model output/last hidden states"""

  inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
  with torch.no_grad():
    last_hidden_state = model(**inputs).last_hidden_state
  return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

emotions_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
emotions_hidden = emotions_encoded.map(extract_hidden_states, batched=True)

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [15]:
emotions_hidden

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 2000
    })
})

In [16]:
import numpy as np
X_train = np.array(emotions_hidden['train']['hidden_state'])
X_valid = np.array(emotions_hidden["validation"]["hidden_state"])
y_train = np.array(emotions_hidden["train"]["label"])
y_valid = np.array(emotions_hidden["validation"]["label"])
X_train.shape, X_valid.shape

((16000, 768), (2000, 768))

In [17]:
y_train.shape, y_valid.shape

((16000,), (2000,))

In [19]:
import pandas as pd
train = pd.concat([pd.DataFrame(X_train), pd.Series(y_train)], axis=1)
valid = pd.concat([pd.DataFrame(X_valid), pd.Series(y_valid)], axis=1)

In [20]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter=3000)
lr_clf.fit(X_train, y_train)
lr_clf.score(X_valid, y_valid)

0.6335

In [21]:
class MyDataset(torch.utils.data.Dataset):

  def __init__(self, df):
    df=df

    x = df.iloc[:,0:768].values
    y = df.iloc[:,768].values

    self.x_train = torch.tensor(x, dtype=torch.float32)
    self.y_train = torch.tensor(y, dtype=torch.int32)

  def __len__(self):
    return len(self.y_train)
  
  def __getitem__(self, idx):
    return self.x_train[idx], self.y_train[idx]

In [23]:
#using FFNN to classify
import pandas as pd
import torch.nn as nn                       #ffnn, cnn, rnn etc
import torch.optim as optim                 #optimisers like adam, sgd etc
import torch.nn.functional as f             #activ funcs
from torch.utils.data import DataLoader     #minibatch train test etc
import torchvision.datasets as datasets     #std datasets
import torchvision.transforms as transforms #transformations to perform on dataset
import torch

class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()

    #input layer - hidden layer - output layer; This is our network structure
    self.fc1 = nn.Linear(in_features=input_size, out_features=256)
    self.fc2 = nn.Linear(in_features=256, out_features=128)
    self.fc3 = nn.Linear(128, 64)
    #self.fc4 = nn.Linear(64, 16)
    self.fc5 = nn.Linear(64, num_classes)
    self.Dropout = nn.Dropout(p=0.1)

  def forward(self, x): 
    x = self.Dropout(f.relu(self.fc1(x)))
    x = self.Dropout(f.relu(self.fc2(x)))
    x = self.Dropout(f.relu(self.fc3(x)))
    #x = self.Dropout(f.relu(self.fc4(x)))
    x = (f.relu(self.fc5(x)))
    return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 768
num_classes = pd.Series(emotions['train']['label']).unique().size
learning_rate = .001
batch_size = 64
epochs = 3

train_set = MyDataset(train)
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_set = MyDataset(valid)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

model = NN(input_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(epochs):
  for batch_num, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)
    targets = targets.to(torch.int64)
    scores = model(data) 
    loss = criterion(scores, targets)
    
    #backward
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step() 

def check_accuracy(loader, model):
  '''if loader.dataset.train:
    print("checking accuracy on train data")
  else:
    print("checking accuracy on test data")'''

  num_correct = 0
  num_samples = 0
  model.eval() #some training only operations turned off like dropout etc

  with torch.no_grad():
    for x, y in loader: #loops over batches in a loader
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x) #64*10
      _, predictions = scores.max(1) #probs, indices
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)
    print(f'Accuracy {float(num_correct)/float(num_samples)*100:.2f}')
  model.train() #again puts it in train mode just in case 
  

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Accuracy 59.25
Accuracy 58.45


### This bad accuracy could be because our dataset is highly imbalanced. there are obviously many ways to improve upon this, but our goal here is to get hands on with the library so leaving it till here.